In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Just some utility functions to clean data

In [ ]:
def fill_missing_values(df_data):
    """Fill missing values in data frame, in place."""
    ##########################################################
    df_data.fillna(method="ffill", inplace=True)
    df_data.fillna(method="bfill", inplace=True)
    ##########################################################

In [ ]:
def symbol_to_path(symbol, base_dir="../input"):
    """Return CSV file path given ticker symbol."""
    return os.path.join(base_dir, "gbpusd-price-and-tick-volume/2021.6.16{}-TICK-NoSession.csv".format(str(symbol)))

In [ ]:
def get_data(symbols, dates):
    """Read stock data (adjusted close) for given symbols from CSV files."""
    df_final = pd.DataFrame(index=dates)
    for symbol in symbols:
        file_path = symbol_to_path(symbol)
        df_temp = pd.read_csv(file_path, parse_dates=True, index_col="DateTime",
            usecols=["DateTime", "Ask"], na_values=["nan"])
        df_temp = df_temp['Ask'].resample('15Min').mean()
        df_temp = pd.DataFrame(df_temp,index=df_temp.index)
        df_temp = df_temp.rename(columns={"Ask": symbol})
        #df_final.index.name = "DateTime"
        df_final = df_final.join(df_temp)
        df_final = df_final.dropna(subset=["GBPUSD"])
    return df_final

# Function to plot data about a commodity
In this case to plot GBPUSD data

In [ ]:
def plot_data(df_data,ltitle="GBPUSD Data",ylabel="Date",xlabel="Price"):
    """Plot stock data with appropriate axis labels."""
    ax = df_data.plot(title=ltitle, fontsize=12)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()

The next functions calculate daily returns on a Pandas DataFrame

In [ ]:
def compute_daily_returns(df):
    daily_returns = df.copy()
    daily_returns[1:] = (df[1:]/df[:-1].values) - 1
    daily_returns.iloc[0,:] = 0
    return daily_returns

Next function execute all the steps to plot and calculate some statistics
1. Plot currency values by 15 minuts
2. Calculate Daily returns and plot it
3. calculate Standard Deviation and Mean
4. Organize Daily returns on a historgram and plot mean and std dev. Lines.

In [ ]:
def test_run():
    """Function called by Test Run."""
    # Read data
    symbol_list = ["GBPUSD"]  # list of symbols
    start_date = '2021-05-16 21:00:00'
    end_date = '2021-06-15 00:00:00'
    dates = pd.date_range(start=start_date, end=end_date, freq='15min')  # date range as index pd.date_range(start='2021-05-16 00:00:00.000',end='2021-06-15 00:00:00.000',freq='15min')
    df_data = get_data(symbol_list, dates)  # get data for each symbol

    # Fill missing values
    fill_missing_values(df_data)
    # Plot
    plot_data(df_data)
    
    daily_returns = compute_daily_returns(df_data)
    
    plot_data(daily_returns,ltitle="Daily Returns",ylabel="Daily Returns",xlabel="Date")
    
    daily_returns.hist(bins=20)
    mean = daily_returns['GBPUSD'].mean()
    std = daily_returns['GBPUSD'].std()
    print("Mean is {}, standard deviation is: {}".format(mean,std))
    plt.axvline(mean,color='w',linestyle="dashed",linewidth='2')
    plt.axvline(std,color='r',linestyle="dashed",linewidth='2')
    plt.axvline(-std,color='r',linestyle="dashed",linewidth='2')
    print("\n\n")
    plt.show()
    
    
test_run()